In [224]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from webdriver_manager.chrome import ChromeDriverManager

In [225]:
url = r'https://natgeotraveller.in/author/lakshmi-sankaran/'
driver = webdriver.Chrome(ChromeDriverManager().install())

In [226]:
driver.get(url) 

In [229]:
soup = BeautifulSoup(driver.page_source,'html.parser')
all_posts_div = soup.find(id='articlePostByAuthor')
posts_elements = all_posts_div.find_all("div", {"class": "similar-article-box"})

In [231]:
posts_links= [elem.find('a')['href'] for elem in posts_elements]
print(len(posts))
posts_links[:5]

29


['http://natgeotraveller.in/latest-in-travel-flights-from-pune-to-singapore-quarantine-free-travel-to-taiwan-and-more/',
 'http://natgeotraveller.in/latest-in-travel-schengen-digital-visa-for-smoother-eu-trips/',
 'http://natgeotraveller.in/a-culture-ride-through-chiang-mai-and-chiang-rai/',
 'http://natgeotraveller.in/splendour-at-sea/',
 'http://natgeotraveller.in/in-thailand-nostalgia-on-wheels/']

In [213]:
to_remove = ['\xa0','***']
subscribe_lines = ['To read more stories on travel, cities, food, nature, and adventure',
                   'To\xa0subscribe\xa0to\xa0National Geographic Traveller']
def get_article_text(link):
    driver.get(link)
    soup = BeautifulSoup(driver.page_source,'html.parser').select(".blog-cont")[0]
    div = BeautifulSoup(str(soup),'html.parser')
    text = [i.text for i in div.select("p:not(.vital *):not(.wp-caption-text)")]
    for i in to_remove:
        while True:
            try:
                text.remove(i)
            except ValueError:
                break
    for i in subscribe_lines:
        if(text[-1].startswith(i)):
            del text[-1]
            break
    return " ".join(text)

In [217]:
all_articles = [get_article_text(link) for link in posts_links]

In [218]:
len(all_articles)

29

In [234]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [236]:
vectorizer = TfidfVectorizer(binary=True)

In [237]:
vectorizer.fit(all_articles)

TfidfVectorizer(binary=True)

In [261]:
vectorize = lambda text: vectorizer.transform(text).toarray()
similarity = lambda vec1, vec2: cosine_similarity(vec1, vec2)[0][0]

In [255]:
vectors = [vectorize([text]) for text in all_articles]

In [290]:
num_articles = len(all_articles)
similarity_matrix = [[0 for i in range(num_articles)] for k in range(num_articles)]
for i in range(num_articles):
    for j in range(num_articles):
#         print("{},{}={:.3f}".format(i,j,similarity(vectors[i],vectors[j])))
        similarity_matrix[i][j] = similarity(vectors[i],vectors[j])

In [294]:
import matplotlib.pyplot as plt
import numpy as np